In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts

In [33]:
%matplotlib qt

# Read the data

In [4]:
ds_salaries = pd.read_csv('data/ds_salaries.csv')
ds_salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


# Splitting the data into training and testing sets

In [34]:
X = pd.get_dummies(ds_salaries.drop(labels=["job_title"], axis=1)) # select the features
y = ds_salaries["job_title"] # select the target                                     

# split the data into training and testing sets
X_train, X_test, y_train, y_test = tts(X.to_numpy(), y.to_numpy(), test_size=.10, random_state=10) 
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3379, 185), (3379,), (376, 185), (376,))

# Training the model

In [41]:
def plot_performance(n):
    """Plot the performance of the model for up to n iterations."""
    fig, ax = plt.subplots()
    for i in range(n): 
        model = LogisticRegression(max_iter=n).fit(X_train, y_train)
        score = model.score(X_test, y_test)
        plt.scatter(i, score, c="red")

In [ ]:
plot_performance(100)

: 